In [ ]:
import matplotlib.pyplot as plt
import parse_access_om2_results as par
import plot_pandas_data as ppd
import plot_right_axis_time as pra
from convert_time import days_to_gregorian_years, gregorian_years_to_days

In [ ]:
df = par.parse_om2_dataframe("../access-om2/archive")

In [ ]:
bw = lambda d: d[d["nodesize"] == 28]
cl = lambda d: d[d["nodesize"] == 48]

In [ ]:
df_bw = bw(df)
df_cl = cl(df)

In [ ]:
ocean_cice_ratio_middle = 7

In [ ]:
hi_ice = lambda d: d[d["ocean_ncpus"] / d["ice_ncpus"] <= ocean_cice_ratio_middle]
lo_ice = lambda d: d[d["ocean_ncpus"] / d["ice_ncpus"] > ocean_cice_ratio_middle]

In [ ]:
df_bw_hi = hi_ice(df_bw)
df_bw_lo = lo_ice(df_bw)
df_cl_hi = hi_ice(df_cl)
df_cl_lo = lo_ice(df_cl)

In [ ]:
const_ice_nbr_blocks = 5767

In [ ]:
const_blocks = lambda d: d[d["ice_nbr_blocks"] == const_ice_nbr_blocks]
var_blocks =   lambda d: d[d["ice_nbr_blocks"] != const_ice_nbr_blocks]

In [ ]:
lo_ice_const_blocks = const_blocks(df_cl_lo)
lo_ice_var_blocks = var_blocks(df_cl_lo)

In [ ]:
max_allowed_init_time = 300.0

In [ ]:
fast_init = lambda d: d[d["Initialization"] <= max_allowed_init_time]
slow_init = lambda d: d[d["Initialization"] > max_allowed_init_time]

In [ ]:
df_cl_fast_init = fast_init(df_cl)
df_cl_slow_init = slow_init(df_cl)

In [ ]:
def plot_ice_block_comparison(plot_fn, x, data,
        xticks=[],
        right_axis_fn=pra.no_right_axis,
        fig_nbr=0,
        caption=None,
        **kwargs):                      
    if xticks == []:
        xticks = data[x].sort_values().unique()
    xtick_labels = [str(int(val)) for val in xticks]
    fig, ax = plt.subplots()
    plots = [
        plot_fn(
            hi_ice(data),
            ax=ax,
            xticks=xticks,
            right_axis_fn=right_axis_fn,
            fig_nbr=fig_nbr,
            caption=caption,
            **kwargs)]
    for dataframe in [
        const_blocks(lo_ice(data)),
        var_blocks(lo_ice(data))]:
        plots.append(plot_fn(
            dataframe,
            ax=ax,
            xticks=xticks,
            right_axis_fn=pra.no_right_axis,
            **kwargs))
    legend = (
            "High CICE cores",
            "Low CICE cores, const #blocks",
            "Low CICE cores, var #blocks")
    plt.legend((plot[0] for plot in plots), legend)

In [ ]:
def loglog_om2_speed_by_ncpus(df, **kwargs):
    p = ppd.loglog("om2_ncpus", "OM2 speed",
        data=df,
        xlabel="Number of ACCESS-OM2 cores",
        ylabel="ACCESS-OM2 speed (d/d)",
        **kwargs)
    return p

In [ ]:
ppd.plot_by_xy_labels_fn(
    loglog_om2_speed_by_ncpus,
    "om2_ncpus",
    data=df_bw,
    yticks=[200,500,1000],
    right_yticks=[1,2],
    right_axis_fn=pra.plot_right_axis_yd)

In [ ]:
fig_nbr = 4

In [ ]:
fig_nbr += 1
ppd.plot_by_xy_labels_fn(
    loglog_om2_speed_by_ncpus,
    "om2_ncpus",
    data=df_cl_hi,
    yticks=[500,1000,2000],
    right_yticks=[1,2,3,5],
    right_axis_fn=pra.plot_right_axis_yd,
    fig_nbr=fig_nbr,
    caption="ACCESS-OM2 with current proportion of CICE cores:\n"
            "Simulation speed from walltime")

In [ ]:
def semilogx_om2_speed_per_cpu_by_ncpus(df, **kwargs):
    p = ppd.semilogx("om2_ncpus", "OM2 speed per cpu",
        data=df,
        xlabel="Number of ACCESS-OM2 cores",
        ylabel="ACCESS-OM2 speed (d/d) per core",
        **kwargs)
    return p

In [ ]:
fig_nbr += 1
ppd.plot_by_xy_labels_fn(
    semilogx_om2_speed_per_cpu_by_ncpus,
    "om2_ncpus", 
    data=df_cl_hi,
    yticks=[0.05 * n for n in range(5)],
    right_yticks=[0.0001 * n for n in range(6)],
    right_axis_fn=pra.plot_right_axis_yd,
    fig_nbr=fig_nbr,
    caption="ACCESS-OM2 with current proportion of CICE cores:\n"
            "simulation speed per OM2 core")

In [ ]:
def loglog_ocean_speed_by_ncpus(df, **kwargs):
    p = ppd.loglog("ocean_ncpus", "Ocean speed",
        data=df,
        xlabel="Number of MOM cores",
        ylabel="Ocean loop speed (d/d)",
        **kwargs) 
    return p

In [ ]:
fig_nbr += 1
ppd.plot_by_xy_labels_fn(
    loglog_ocean_speed_by_ncpus,
    "ocean_ncpus", 
    data=df_cl_hi,
    yticks=[500,1000,2000,5000],
    right_yticks=[1,2,5,10],
    right_axis_fn=pra.plot_right_axis_yd,
    marker='d',
    fig_nbr=fig_nbr,
    caption="ACCESS-OM2 with current proportion of CICE cores:\n"
            "Simulation speed from Ocean loop time by number of MOM cores")

In [ ]:
def semilogx_ocean_speed_per_cpu_by_ncpus(df, **kwargs):
    p = ppd.semilogx("ocean_ncpus", "Ocean speed per cpu",
        data=df,
        xlabel="Number of MOM cores",
        ylabel="Ocean loop speed (d/d) per core",
        **kwargs) 
    return p

In [ ]:
fig_nbr += 1
ppd.plot_by_xy_labels_fn(
    semilogx_ocean_speed_per_cpu_by_ncpus,    
    "ocean_ncpus",
    data=df_cl_hi,
    yticks=[n/20.0 for n in range(7)],
    right_yticks=[n/5000.0 for n in range(5)],
    right_axis_fn=pra.plot_right_axis_yd,
    marker='d',
    fig_nbr=fig_nbr,
    caption="ACCESS-OM2 with current proportion of CICE cores:\n"
            "Ocean loop speed per OM2 core by number of MOM cores")

In [ ]:
def loglog_ice_speed_by_ncpus(df, **kwargs):
    p = ppd.loglog("ice_ncpus", "Ice TimeLoop speed",
        data=df,
        xlabel="Number of CICE cores",
        ylabel="Ice loop speed (d/d)",
        **kwargs) 
    return p

In [ ]:
fig_nbr += 1
ppd.plot_by_xy_labels_fn(
    loglog_ice_speed_by_ncpus, 
    "ice_ncpus", 
    data=df_cl_hi,
    yticks=[500,1000,2000],
    right_yticks=[1,2,5,10,20],
    right_axis_fn=pra.plot_right_axis_yd,
    marker='*',
    fig_nbr=fig_nbr,
    caption="ACCESS-OM2 with current proportion of CICE cores:\n"
            "Simulation speed from Ice loop time by number of CICE cores")

In [ ]:
def semilogx_ice_speed_per_cpu_by_ncpus(df, **kwargs):
    p = ppd.semilogx("ice_ncpus", "Ice speed per cpu",
        data=df,
        xlabel="Number of CICE cores",
        ylabel="Ice loop speed (d/d) per core",
        **kwargs) 
    return p

In [ ]:
fig_nbr += 1
ppd.plot_by_xy_labels_fn(
    semilogx_ice_speed_per_cpu_by_ncpus,
    "ice_ncpus",
    data=df_cl_hi,
    yticks=[n/5.0 for n in range(8)],
    right_yticks=[n/1000.0 for n in range(8)],
    right_axis_fn=pra.plot_right_axis_yd,
    marker='*',
    fig_nbr=fig_nbr,
    caption="ACCESS-OM2 with current proportion of CICE cores:\n"
            "Ice loop speed per OM2 core by number of CICE cores")

In [ ]:
def loglog_ice_speed_by_ocean_ncpus_nbr_blocks(df, **kwargs):
    p = ppd.loglog_by_group("ice_nbr_blocks", "Ice TimeLoop speed",
        by="ocean_ncpus",
        data=df,
        xlabel="Number of CICE blocks",
        ylabel="Ice loop speed (d/d)",
        legend_title="MOM cores",
        **kwargs)
    return p

In [ ]:
fig_nbr += 1
ppd.plot_by_xy_labels_fn(
    loglog_ice_speed_by_ocean_ncpus_nbr_blocks,
    "ice_nbr_blocks",  
    data=df_cl_hi,
    yticks=[500,1000,2000,3000],
    right_yticks=[1,2,4,8],
    right_axis_fn=pra.plot_right_axis_yd,
    marker='*',
    fig_nbr=fig_nbr,
    caption="ACCESS-OM2 with current proportion of CICE cores:\n"
            "Effect on Ice loop speed of varying number of CICE blocks")

In [ ]:
ppd.plot_by_xy_labels_fn(
    loglog_ice_speed_by_ocean_ncpus_nbr_blocks,
    "ice_nbr_blocks", 
    data=df_cl_lo,
    yticks=[500,1000,2000,3000],
    right_yticks=[1,2,4,8],
    right_axis_fn=pra.plot_right_axis_yd,
    marker='*')

In [ ]:
fig_nbr += 1
plot_ice_block_comparison(
    loglog_om2_speed_by_ncpus,
    "om2_ncpus",
    df_cl,
    yticks=[500,1000,2000],
    right_yticks=[1,2,3,5],
    right_axis_fn=pra.plot_right_axis_yd,
    fig_nbr=fig_nbr,
    caption="ACCESS-OM2: Effect on simulation speed from walltime:\n"
            "Varying the proportion of CICE cores and number of CICE blocks")

In [ ]:
fig_nbr += 1
plot_ice_block_comparison(
    semilogx_om2_speed_per_cpu_by_ncpus,
    "om2_ncpus",
    df_cl,
    yticks=[0.05 * n for n in range(5)],
    right_yticks=[0.0001 * n for n in range(6)],
    right_axis_fn=pra.plot_right_axis_yd,
    fig_nbr=fig_nbr,
    caption="ACCESS-OM2: Effect on simulation speed per OM2 core:\n"
            "Varying the proportion of CICE cores and number of CICE blocks")

In [ ]:
def loglog_om2_speed_by_ocean_ncpus_ice_ncpus(df, **kwargs):
    p = ppd.loglog_by_group("ocean_ncpus", "OM2 speed",
        by="ice_ncpus",
        data=df,
        xlabel="Number of MOM cores",
        ylabel="ACCESS-OM2 speed (d/d)",
        legend_title="CICE cores",
        **kwargs)
    return p

In [ ]:
fig_nbr += 1
ppd.plot_by_xy_labels_fn(
    loglog_om2_speed_by_ocean_ncpus_ice_ncpus,
    "ocean_ncpus",  
    data=const_blocks(df_cl),
    yticks=[500,1000,2000],
    right_yticks=[1,2,3,5],
    right_axis_fn=pra.plot_right_axis_yd,
    marker='+',
    fig_nbr=fig_nbr,
    caption="ACCESS-OM2: Effect on simulation speed:\n"
            "Varying the number of MOM cores, keeping CICE cores and blocks constant")

In [ ]:
def semilogx_om2_speed_per_cpu_by_ocean_ncpus_ice_ncpus(df, **kwargs):
    p = ppd.semilogx_by_group("ocean_ncpus", "OM2 speed per cpu",
        by="ice_ncpus",
        data=df,
        xlabel="Number of MOM cores",
        ylabel="ACCESS-OM2 speed (d/d) per core",
        legend_title="CICE cores",
        **kwargs)
    return p

In [ ]:
fig_nbr += 1
ppd.plot_by_xy_labels_fn(
    semilogx_om2_speed_per_cpu_by_ocean_ncpus_ice_ncpus,
    "ocean_ncpus",  
    data=const_blocks(df_cl),
    yticks=[0.05 * n for n in range(5)],
    right_yticks=[0.0001 * n for n in range(6)],
    right_axis_fn=pra.plot_right_axis_yd,
    marker='+',
    fig_nbr=fig_nbr,
    caption="ACCESS-OM2: Effect on simulation speed per OM2 core:\n"
            "Varying the number of MOM cores, keeping CICE cores and blocks constant")

In [ ]:
fig_nbr += 1
plot_ice_block_comparison(
    loglog_ocean_speed_by_ncpus,
    "ocean_ncpus",
    df_cl,
    yticks=[500,1000,2000,5000],
    right_yticks=[1,2,5,10],
    right_axis_fn=pra.plot_right_axis_yd,
    marker='d',
    fig_nbr=fig_nbr,
    caption="ACCESS-OM2: Effect on Ocean loop speed by number of MOM cores:\n"
            "Varying the proportion of CICE cores and number of CICE blocks")

In [ ]:
fig_nbr += 1
plot_ice_block_comparison(
    semilogx_ocean_speed_per_cpu_by_ncpus,
    "ocean_ncpus",
    df_cl,
    yticks=[n/20.0 for n in range(7)],
    right_yticks=[n/5000.0 for n in range(5)],
    right_axis_fn=pra.plot_right_axis_yd,
    marker='d',
    fig_nbr=fig_nbr,
    caption="ACCESS-OM2: Effect on Ocean loop speed per MOM core:\n"
            "Varying the proportion of CICE cores and number of CICE blocks")    

In [ ]:
fig_nbr += 1
plot_ice_block_comparison(
    loglog_ice_speed_by_ncpus,
    "ice_ncpus", 
    df_cl,
    yticks=[500,1000,2000],
    right_yticks=[1,2,5,10],
    right_axis_fn=pra.plot_right_axis_yd,
    marker='*',
    fig_nbr=fig_nbr,
    caption="ACCESS-OM2: Effect on Ice loop speed by number of CICE cores:\n"
            "Varying the proportion of CICE cores and number of CICE blocks")

In [ ]:
fig_nbr += 1
plot_ice_block_comparison(
    semilogx_ice_speed_per_cpu_by_ncpus,
    "ice_ncpus", 
    df_cl,
    yticks=[0,0.5,1,1.5,2],
    right_yticks=[n/1000.0 for n in range(6)],
    right_axis_fn=pra.plot_right_axis_yd,
    marker='*',
    fig_nbr=fig_nbr,
    caption="ACCESS-OM2: Effect on Ice loop speed per CICE core:\n"
            "Varying the proportion of CICE cores and number of CICE blocks")

In [ ]:
def get_mean(df, x, y):
    mean_y_by_x=dict()
    group_ncpus=df.groupby(x)
    for name, group in group_ncpus:
        means = group.groupby(x)[y].mean()
        mean_y_by_x[name] = means[name]
    return mean_y_by_x

In [ ]:
def round_dict(d, n):
    return {k: round(v, n) for k, v in d.items()}

In [ ]:
def days_to_years_dict(d):
    return {k: days_to_gregorian_years(v) for k, v in d.items()}

In [ ]:
def div_dicts(a, b, rounding=3):
    a_keys = a.keys()
    b_keys = b.keys()
    common_keys = sorted(list(set(a_keys) & set(b_keys)))
    return dict([(k, round(a[k]/b[k], rounding)) for k in common_keys])

In [ ]:
print("ACCESS-OM2 speed per core by OM2 cores")
print("High")
high_mean = get_mean(df_cl_hi,"om2_ncpus","OM2 speed per cpu")
print("d/d")
print(round_dict(high_mean,3))
print("Y/d")
print(round_dict(days_to_years_dict(high_mean), 5))                        
print("Const")
low_const_mean = get_mean(lo_ice_const_blocks,"om2_ncpus","OM2 speed per cpu")
print("d/d")
print(round_dict(low_const_mean,3))
print("Y/d")
print(round_dict(days_to_years_dict(low_const_mean), 5))                        
print("Var")
low_var_mean = get_mean(lo_ice_var_blocks,"om2_ncpus","OM2 speed per cpu")
print("d/d")
print(round_dict(low_var_mean,3))
print("Y/d")
print(round_dict(days_to_years_dict(low_var_mean), 5))                        

In [ ]:
print("ACCESS-OM2 speed by MOM cores")
print("High")
high_mean = get_mean(df_cl_hi,"ocean_ncpus","OM2 speed")
print("d/d")
print(round_dict(high_mean,3))
print("Y/d")
print(round_dict(days_to_years_dict(high_mean), 5))                        
print("Const")
low_const_mean = get_mean(lo_ice_const_blocks,"ocean_ncpus","OM2 speed")
print("d/d")
print(round_dict(low_const_mean,3))
print("Y/d")
print(round_dict(days_to_years_dict(low_const_mean), 5))                        
print("Var")
low_var_mean = get_mean(lo_ice_var_blocks,"ocean_ncpus","OM2 speed")
print("d/d")
print(round_dict(low_var_mean,3))
print("Y/d")
print(round_dict(days_to_years_dict(low_var_mean), 5))
print(div_dicts(high_mean, low_const_mean, 2))

In [ ]:
print("ACCESS-OM2 speed per core by MOM cores")
print("High")
high_mean = get_mean(df_cl_hi,"ocean_ncpus","OM2 speed per cpu")
print("d/d")
print(round_dict(high_mean,3))
print("Y/d")
print(round_dict(days_to_years_dict(high_mean), 5))                        
print("Const")
low_const_mean = get_mean(lo_ice_const_blocks,"ocean_ncpus","OM2 speed per cpu")
print("d/d")
print(round_dict(low_const_mean,3))
print("Y/d")
print(round_dict(days_to_years_dict(low_const_mean), 5))                        
print("Var")
low_var_mean = get_mean(lo_ice_var_blocks,"ocean_ncpus","OM2 speed per cpu")
print("d/d")
print(round_dict(low_var_mean,3))
print("Y/d")
print(round_dict(days_to_years_dict(low_var_mean), 5))
print(div_dicts(high_mean, low_const_mean, 2))

In [ ]:
print("Ocean loop speed per core")
print("High")
high_mean = get_mean(df_cl_hi,"ocean_ncpus","Ocean speed per cpu")
print("d/d")
print(round_dict(high_mean,2))
print("Y/d")
print(round_dict(days_to_years_dict(high_mean), 5))                        
print("Const")
low_const_mean = get_mean(lo_ice_const_blocks,"ocean_ncpus","Ocean speed per cpu")
print("d/d")
print(round_dict(low_const_mean,2))
print("Y/d")
print(round_dict(days_to_years_dict(low_const_mean), 5))                        
print("Var")
low_var_mean = get_mean(lo_ice_var_blocks,"ocean_ncpus","Ocean speed per cpu")
print("d/d")
print(round_dict(low_var_mean,2))
print("Y/d")
print(round_dict(days_to_years_dict(low_var_mean), 5))
print(div_dicts(high_mean, low_const_mean, 2))

In [ ]:
print("Ice loop speed per core")
print("High")
high_mean = get_mean(df_cl_hi,"ice_ncpus","Ice speed per cpu")
print("d/d")
print(round_dict(high_mean,2))
print("Y/d")
print(round_dict(days_to_years_dict(high_mean), 4))                        
print("Const")
low_const_mean = get_mean(lo_ice_const_blocks,"ice_ncpus","Ice speed per cpu")
print("d/d")
print(round_dict(low_const_mean,2))
print("Y/d")
print(round_dict(days_to_years_dict(low_const_mean), 4))                        
print("Var")
low_var_mean = get_mean(lo_ice_var_blocks,"ice_ncpus","Ice speed per cpu")
print("d/d")
print(round_dict(low_var_mean,2))
print("Y/d")
print(round_dict(days_to_years_dict(low_var_mean), 4))
print(div_dicts(high_mean, low_const_mean, 2))

In [ ]:
def semilogx_relative_ocean_oasis_recv_by_om2_ncpus(df, **kwargs):
    p = ppd.semilogx("om2_ncpus", "relative ocean_oasis_recv",
        data=df,
        xlabel="Number of ACCESS-OM2 cores",
        ylabel="MOM oasis_recv / Ocean loop time",
        **kwargs) 
    return p

In [ ]:
fig_nbr += 1
plot_ice_block_comparison(
    semilogx_relative_ocean_oasis_recv_by_om2_ncpus,
    "om2_ncpus",
    df_cl,
    yticks=[n/5.0 for n in range(6)],
    marker='d',
    fig_nbr=fig_nbr,
    caption="ACCESS-OM2: Effect on relative MOM wait time:\n"
            "Varying the proportion of CICE cores and number of CICE blocks")

In [ ]:
def semilogx_relative_ice_from_ocn_by_om2_ncpus(df, **kwargs):
    p = ppd.semilogx("om2_ncpus", "relative ice_from_ocn",
        data=df,
        xlabel="Number of ACCESS-OM2 cores",
        ylabel="CICE from_ocn / TimeLoop",
        **kwargs) 
    return p

In [ ]:
fig_nbr += 1
plot_ice_block_comparison(
    semilogx_relative_ice_from_ocn_by_om2_ncpus,
    "om2_ncpus",
    df_cl,
    yticks=[n/10.0 for n in range(5)],
    marker='*',
    fig_nbr=fig_nbr,
    caption="ACCESS-OM2: Effect on relative CICE wait time:\n"
            "Varying the proportion of CICE cores and number of CICE blocks")

In [ ]:
def semilogx_relative_wait_by_om2_ncpus(df, **kwargs):
    p = ppd.semilogx("om2_ncpus", "relative wait",
        data=df,
        xlabel="Number of ACCESS-OM2 cores",
        ylabel="(from_ocn + oasis_recv) / (TimeLoop + Ocean)",
        **kwargs) 
    return p

In [ ]:
fig_nbr += 1
plot_ice_block_comparison(
    semilogx_relative_wait_by_om2_ncpus,
    "om2_ncpus",
    df_cl,
    yticks=[n/10.0 for n in range(4)],
    fig_nbr=fig_nbr,
    caption="ACCESS-OM2: Effect on MOM wait time vs CICE wait time:\n"
            "Varying the proportion of CICE cores and number of CICE blocks")

In [ ]:
def semilogx_walltime_wait_by_om2_ncpus(df, **kwargs):
    p = ppd.semilogx("om2_ncpus", "wait relative to walltime",
        data=df,
        xlabel="Number of ACCESS-OM2 cores",
        ylabel="(from_ocn + oasis_recv) / Walltime",
        **kwargs) 
    return p

In [ ]:
fig_nbr += 1
plot_ice_block_comparison(
    semilogx_walltime_wait_by_om2_ncpus,
    "om2_ncpus",
    df_cl,
    yticks=[n/10.0 for n in range(6)],
    fig_nbr=fig_nbr,
    caption="ACCESS-OM2: Effect on relative total wait time:\n"
            "Varying the proportion of CICE cores and number of CICE blocks")

In [ ]:
def semilogx_om2_init_time_by_om2_ncpus(df, **kwargs):
    p = ppd.semilogx("om2_ncpus", "Initialization",
        data=df,
        xlabel="Number of ACCESS-OM2 cores",
        ylabel="ACCESS-OM2 Initialization time (s)",
        **kwargs) 
    return p

In [ ]:
fig_nbr += 1
plot_ice_block_comparison(
    semilogx_om2_init_time_by_om2_ncpus,
    "om2_ncpus", 
    df_cl,
    yticks=[100*n for n in range(6)],
    right_yticks=[n/50.0 for n in range(8)],
    right_axis_fn=pra.plot_right_axis_sh,
    marker='+',
    fig_nbr=fig_nbr,
    caption="ACCESS-OM2: Effect on initialization time:\n"
            "Varying the proportion of CICE cores and number of CICE blocks")

In [ ]:
fig_nbr += 1
plot_ice_block_comparison(
    semilogx_om2_init_time_by_om2_ncpus,
    "om2_ncpus", 
    df_cl_fast_init,
    yticks=[50*n for n in range(7)],
    right_yticks=[n/50.0 for n in range(5)],
    right_axis_fn=pra.plot_right_axis_sh,
    marker='+',
    fig_nbr=fig_nbr,
    caption="ACCESS-OM2: Effect on initialization time (outliers removed):\n"
            "Varying the proportion of CICE cores and number of CICE blocks")

In [ ]:
def semilogx_ocean_init_time_by_om2_ncpus(df, **kwargs):
    p = ppd.semilogx("om2_ncpus", "(Ocean initialization)",
        data=df,
        xlabel="Number of ACCESS-OM2 cores",
        ylabel="MOM Initialization time (s)",
        **kwargs) 
    return p

In [ ]:
plot_ice_block_comparison(
    semilogx_ocean_init_time_by_om2_ncpus,
    "om2_ncpus", 
    df_cl,
    yticks=[100*n for n in range(6)],
    right_yticks=[n/50.0 for n in range(9)],
    right_axis_fn=pra.plot_right_axis_sh,
    marker='d')

In [ ]:
fig_nbr += 1
plot_ice_block_comparison(
    semilogx_ocean_init_time_by_om2_ncpus,
    "om2_ncpus", 
    df_cl_fast_init,
    yticks=[20*n for n in range(5)],
    right_yticks=[n/100.0 for n in range(4)],
    right_axis_fn=pra.plot_right_axis_sh,
    marker='d',
    fig_nbr=fig_nbr,
    caption="ACCESS-OM2: Effect on MOM initialization time (outliers removed):\n"
            "Varying the proportion of CICE cores and number of CICE blocks")

In [ ]:
def semilogx_ice_init_time_by_om2_ncpus(df, **kwargs):
    p = ppd.semilogx("om2_ncpus", "Ice initialization",
        data=df,
        xlabel="Number of ACCESS-OM2 cores",
        ylabel="CICE initialization time (s)",
        **kwargs) 
    return p

In [ ]:
plot_ice_block_comparison(
    semilogx_ice_init_time_by_om2_ncpus,
    "om2_ncpus", 
    df_cl,
    yticks=[50*n for n in range(7)],
    right_yticks=[n/50.0 for n in range(5)],
    right_axis_fn=pra.plot_right_axis_sh,
    marker='*')

In [ ]:
fig_nbr += 1
plot_ice_block_comparison(
    semilogx_ice_init_time_by_om2_ncpus,
    "om2_ncpus", 
    df_cl_fast_init,
    yticks=[20*n for n in range(7)],
    right_yticks=[n/100.0 for n in range(4)],
    right_axis_fn=pra.plot_right_axis_sh,
    marker='*',
    fig_nbr=fig_nbr,
    caption="ACCESS-OM2: Effect on CICE initialization time (outliers removed):\n"
            "Varying the proportion of CICE cores and number of CICE blocks")

In [ ]:
def loglog_ocean_time_per_step_by_om2_ncpus(df, **kwargs):
    p = ppd.loglog("om2_ncpus", "Ocean per step",
        data=df,
        xlabel="Number of ACCESS-OM2 cores",
        ylabel="Ocean loop time per step (s)",
        **kwargs) 
    return p

In [ ]:
plot_ice_block_comparison(
    loglog_ocean_time_per_step_by_om2_ncpus,
    "om2_ncpus",
    df_cl,
    yticks=[0.1,0.2,0.5,1],
    right_yticks=[0.00005,0.0001,0.0002],
    right_axis_fn=pra.plot_right_axis_sh,
    marker='d')

In [ ]:
def semilogx_ocean_cpu_time_per_step_by_om2_ncpus(df, **kwargs):
    p = ppd.semilogx("om2_ncpus", "ocean_cpu_time per step",
        data=df,
        xlabel="Number of ACCESS-OM2 cores",
        ylabel="MOM OM2 core time per step (core s)",
        **kwargs) 
    return p

In [ ]:
fig_nbr=29

In [ ]:
fig_nbr += 1
plot_ice_block_comparison(
    semilogx_ocean_cpu_time_per_step_by_om2_ncpus,
    "om2_ncpus", 
    df_cl,
    yticks=[],
    right_yticks=[n/10.0 for n in range(8)],
    right_axis_fn=pra.plot_right_axis_sh,
    marker='d',
    fig_nbr=fig_nbr,
    caption=r"ACCESS-OM2: Ocean loop time per step $\times$ number of OM2 cores:"
           "\nVarying the proportion of CICE cores and number of CICE blocks")    

In [ ]:
def loglog_ice_time_per_step_by_om2_ncpus(df, **kwargs):
    p = ppd.loglog("om2_ncpus", "Ice TimeLoop per step",
        data=df,
        xlabel="Number of ACCESS-OM2 cores",
        ylabel="Ice loop time per step (s)",
        **kwargs) 
    return p

In [ ]:
plot_ice_block_comparison(
    loglog_ice_time_per_step_by_om2_ncpus,
    "om2_ncpus",
    df_cl,
    yticks=[0.1,0.2,0.5,1],
    right_axis_fn=pra.no_right_axis,
    marker='*')

In [ ]:
def semilogx_ice_cpu_time_per_step_by_om2_ncpus(df, **kwargs):
    p = ppd.semilogx("om2_ncpus", "ice_cpu_time per step",
        data=df,
        xlabel="Number of ACCESS-OM2 cores",
        ylabel="CICE OM2 core time per step (core s)",
        **kwargs) 
    return p

In [ ]:
fig_nbr += 1
plot_ice_block_comparison(
    semilogx_ice_cpu_time_per_step_by_om2_ncpus,
    "om2_ncpus", 
    df_cl,
    yticks=[100*n for n in range(7)],
    right_yticks=[n/50.0 for n in range(9)],
    right_axis_fn=pra.plot_right_axis_sh,
    marker='*',
    fig_nbr=fig_nbr,
    caption=r"ACCESS-OM2: Ocean loop time per step $\times$ number of OM2 cores:"
           "\nVarying the proportion of CICE cores and number of CICE blocks")